# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        3.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        0.0  2.0  0.0  4.0  0.0  4.0  4.0  0.0  0.0  2.0  ...  0.0  4.0  4.0   
 2        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 3        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 4        0.0  4.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  4.0  ...  0.0  0.0  3.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      4.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 296      0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 297      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  3.0  4.0  0.0   
 298      0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  5.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 

LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ETA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

from math import sqrt

# create data objects 
train_dataframe = pd.DataFrame(imputed_train_ds, copy=True)
normalized_ratings_user = train_dataframe.copy()
normalized_ratings_item = train_dataframe.copy()
np_train = train_dataframe.to_numpy()

# creates a binary dataframe by dividing values by themselves if above 0
binary_train = train_dataframe.applymap(lambda x: x/x if x > 0 else x*x)

# calculates average ratings
user_average_rating = np_train.sum(axis=1)/binary_train.sum(axis=1)
item_average_rating = np_train.sum(axis=0)/binary_train.sum(axis=0)

# calculates weighted similarity rating
def similarity_rating(x,y, constant, N):
    top_sum = []
    bottom_sum_i = []
    bottom_sum_j = []
    
    # calculates common ratings
    def shared_ratings(i, j, count=0):
        if count == N:
            return
        elif i[count] != 0 and j[count] != 0:
            sum_ij = i[count]*j[count]
            top_sum.append(sum_ij)
            bottom_sum_i.append(i[count])
            bottom_sum_j.append(j[count])
            shared_ratings(i,j, count=count+1)
        else:
            shared_ratings(i,j, count=count+1)
        
    shared_ratings(x, y)   
    top_sum = sum(top_sum)
    bottom_i = sqrt(sum([i**2 for i in bottom_sum_i]))
    bottom_j = sqrt(sum([j**2 for j in bottom_sum_j]))
    pearson = top_sum/(bottom_i * bottom_j + EPSILON)
    return ((min(len(bottom_sum_i), constant))/constant) * pearson

# normalize the data (value - mean)
for i in range(300):
    normalized_ratings_user.iloc[i] = normalized_ratings_user.iloc[i].apply(
                                        lambda j: j-user_average_rating[i] if j>0 else j+0)

for i in range(500):
    normalized_ratings_item.iloc[:,i] = normalized_ratings_item.iloc[:,i].apply(
                                        lambda x: x-item_average_rating[i] if i>0 else i+0)

user_similarity_matrix = np.zeros((300,300))     
item_similarity_matrix = np.zeros((500,500))

# calculate and append item similarity score to item matrix
for i in range(500):
    print('\ritem similarity matrix: ' + str(round((i/500)*100, 2)) +'% complete', end='')
    for j in range(500):
        if i == j:
            pass
        else:
            item_similarity_matrix[i][j] = similarity_rating( \
                normalized_ratings_item.iloc[:,i], normalized_ratings_item.iloc[:,j], DELTA, 299)

# calculate and append user similarity score to user matrix
for i in range(300):
    print('\ruser similarity matrix: ' + str(round((i/300)*100, 2)) + '% complete', end='')
    for j in range(300):
        if i == j:
            pass
        else:
            user_similarity_matrix[i][j] = similarity_rating( \
                normalized_ratings_user.iloc[i], normalized_ratings_user.iloc[j], GAMMA, 499)

user similarity matrix: 99.67% complete

In [8]:
user_df = pd.DataFrame(user_similarity_matrix)
item_df = pd.DataFrame(item_similarity_matrix)

# creates a nested dictionary of similar users given a single user (Sim > ETA)
def similar_users(user):
    sim_list = []
    sim_index = []
    mask = user_df.iloc[user]
    for i in range(len(mask)):
        if mask[i] > ETA:
            sim_list.append(mask[i])
            sim_index.append(i-1)
        else:
            pass
    return dict(zip(sim_index, sim_list))

# creates a nested dictionary of similar items given a single item (Sim > THETA)
def similar_items(item):
    item_list = []
    item_index = []
    mask = item_df.iloc[item]
    
    for i in range(len(mask)):
        if mask[i] > THETA:
            item_list.append(mask[i])
            item_index.append(i)
        else:
            pass
    return dict(zip(item_index, item_list))

sim_user_dict = {}
sim_item_dict = {}

# these functions collate similar users/items into one nested dictionary
for i in range(500):
    sim_item_dict[i] = similar_items(i)

for i in range(300):
    sim_user_dict[i] = similar_users(i)


In [9]:
# function to process the whole dataframe
def missing_values_prediction(data, out):
    
    # equation 9 from report
    def only_users(user_index, item_index):
        sum_similar_users = []
        sum_sim_with_ratings = []
        
        for i in sim_user_dict[user_index].values():
            sum_similar_users.append(i)
        for key, value in sim_user_dict[user_index].items():
            similarity = value
            rating = data.iloc[key, item_index]
            eq_top = similarity * rating
            sum_sim_with_ratings.append(eq_top)
        sum_similar_users = sum(sum_similar_users)+EPSILON
        sum_sim_with_ratings = sum(sum_sim_with_ratings)
        prediction =user_average_rating[user_index] + (sum_sim_with_ratings/sum_similar_users)
        return prediction
    
    # equation 10 from report
    def only_items(user_index, item_index):
        sum_similar_items = []
        sum_sim_plus_ratings = []
    
        for i in sim_item_dict[item_index].values():
            sum_similar_items.append(i)
            
        for key, value in sim_item_dict[item_index].items():
            similarity = value
            rating = data.iloc[user_index, key]
            eq_top = similarity * rating
            sum_sim_plus_ratings.append(eq_top)
        
        
        sum_similar_items = sum(sum_similar_items)+EPSILON
        sum_sim_plus_ratings = sum(sum_sim_plus_ratings)
        prediction = item_average_rating[item_index] + (sum_sim_plus_ratings/sum_similar_items)
        return prediction
   
    # equation 11 from report
    def null_users_items(user_index, item_index):
        prediction = 0
        return prediction
    
    # equation 8 from report
    def users_and_items(user_index, item_index):
        prediction = (LAMBDA * only_users(user_index, item_index))+ \
                     ((1-LAMBDA)*only_items(user_index, item_index))
        return prediction
    
    # tests whether similar users/items exist for a given user/item, and directs to the appropriate
    # function from the report
    for user in range(300):
        print('\rMissing values prediction: '+str((user/300)*100)+'% complete', end='')
        for item in range(500):
            if data.iloc[user][item] == 0:
                if not any(sim_user_dict[user].values()) and not any(sim_item_dict[item].values()):
                    out[user][item] = null_users_items(user, item)
                elif any(sim_user_dict[user].values()) and not any(sim_item_dict.values()):
                    out[user][item] =  only_users(user, item)
                elif not any(sim_user_dict[user].values()) and any(sim_item_dict.values()):
                    out[user][item] = only_items(user, item)
                elif any(sim_user_dict[user].values()) and any(sim_item_dict.values()):
                    out[user][item] = users_and_items(user, item)
                    
temp = imputed_train_ds.copy()
missing_values_prediction(normalized_ratings_user, temp)



Missing values prediction: 99.66666666666667% completeee

In [10]:
imputed_train_ds = temp

# Evaluation

In [11]:
### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [12]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,3.000000,3.639611,4.257986,3.910932,3.479410,3.255197,3.898395,4.000000,2.992069,3.648810,...,2.991577,2.833558,3.852707,3.838886,2.942275,2.229971,4.000000,3.587126,3.000000,3.068966
1,0.000000,2.000000,4.291771,4.000000,3.426439,4.000000,4.000000,3.644236,2.950128,2.000000,...,2.937255,4.000000,4.000000,3.816170,3.000000,3.000000,1.995041,3.549562,2.839290,3.032040
2,0.000000,3.614951,4.000000,4.000000,4.000000,3.194995,3.877520,4.000000,2.949898,3.648810,...,2.958499,2.817640,3.814100,3.811413,2.901022,2.202534,2.003759,3.554014,2.842264,3.041962
3,4.000000,3.635358,5.000000,3.838011,1.000000,3.221615,3.000000,2.000000,2.963678,3.648810,...,2.983132,2.839243,3.831660,4.000000,1.000000,2.229155,2.024312,3.580671,2.860346,2.000000
4,3.536224,4.000000,3.470153,3.374718,3.241624,3.170208,3.000000,3.323983,3.092923,4.000000,...,3.093887,3.047369,3.000000,3.347975,3.079818,2.866614,1.485390,3.272267,3.058809,3.118162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.000000,3.606571,3.000000,3.813877,3.446690,3.213078,3.864868,3.749184,2.964268,3.648810,...,2.952859,2.805321,3.818406,3.813969,2.909498,2.203368,2.011432,3.557402,2.845739,3.042909
296,5.270532,4.116984,3.898335,5.000000,4.069801,4.000024,4.486027,4.436320,2.809660,4.413562,...,3.920834,2.764884,4.179491,4.464349,3.906006,3.693157,3.635500,4.099650,3.885441,3.945770
297,5.017522,1.000000,3.733663,3.588184,3.489985,3.406959,3.618718,3.555579,3.345780,3.560552,...,3.000000,4.000000,3.605800,3.602849,3.334318,3.120168,3.062121,3.525856,3.311995,3.373172
298,0.000000,3.626030,4.263028,3.000000,3.462226,3.232139,3.904725,4.000000,2.984710,3.648810,...,2.980021,2.832778,5.000000,3.834968,2.938525,3.000000,2.041199,3.583167,2.868084,3.062397


In [13]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[ 5.85407503e-02,  5.48653444e-01,  2.31528108e-01, ...,
        -1.16951931e-04,  4.83646295e-01,  3.03487967e-03],
       [ 3.54593938e-01,  4.26647348e-02,  1.93358784e-01, ...,
         2.19968261e-01,  1.22883012e-01,  2.49124791e-01],
       [ 1.94995588e-01, -1.25920842e-01,  5.33115487e-02, ...,
         2.32879460e-01, -2.11639808e-01,  1.47427713e-01],
       ...,
       [ 4.80077187e-01,  3.43105467e-01,  4.44370844e-01, ...,
         1.18795451e-01, -1.71968431e-01,  3.36967369e-01],
       [-1.87819414e-02,  2.34728614e-01, -2.54673693e-02, ...,
         4.40982878e-01,  1.33216241e-01, -1.03795975e-01],
       [ 4.00603733e-04,  4.43756620e-02, -4.39043896e-03, ...,
         4.54009069e-02,  7.50890401e-02, -1.03179764e-01]])

## Predict Ratings of Testing Set

In [14]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 3.79586573, ..., 3.09264072, 0.        ,
        0.        ],
       ...,
       [0.        , 3.57396318, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [4.13264344, 0.        , 4.10833994, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [15]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7456084428528457, RMSE: 0.9638337252408127
